In [116]:
import pandas as pd
import re
import os
from google.cloud import storage
import numpy as np

# Nếu bạn có service account key file, uncomment dòng dưới:
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'path/to/your/service-account-key.json'

df = pd.read_csv("../data/batdongsan1.csv")


In [117]:
df = df.rename(columns={
    'Diện tích sử dụng' : 'dien_tich_su_dung_m2',
    'Mã BĐS' : 'ma_bds',
    'price' : 'gia_ty',
    'Pháp lý' : 'phap_ly',
    'Ngày đăng' : 'ngay_dang',
    'Nhà tắm'  : 'nha_tam',
    'Phòng ngủ': 'phong_ngu',
    'address': 'dia_chi',
    'Diện tích đất': 'dien_tich_dat'
})

In [118]:

def tach_dien_tich_va_kich_thuoc(text):
   
    if pd.isna(text) or not isinstance(text, str):
        return {'dien_tich': None, 'chieu_dai': None, 'chieu_rong': None}
    
    text = str(text).strip()
    
    dien_tich_match = re.search(r'(\d+(?:[.,]\d+)?)\s*m2', text, re.IGNORECASE)
    dien_tich = None
    if dien_tich_match:
        dien_tich = float(dien_tich_match.group(1).replace(',', '.'))
    
    kich_thuoc_match = re.search(r'\(([^)]+)\)', text)
    chieu_dai = None
    chieu_rong = None
    
    if kich_thuoc_match:
        kich_thuoc_str = kich_thuoc_match.group(1)
        numbers = re.findall(r'(\d+(?:[.,]\d+)?)', kich_thuoc_str)
        
        if len(numbers) >= 2:
            numbers_float = [float(num.replace(',', '.')) for num in numbers]
            chieu_dai = max(numbers_float)
            chieu_rong = min(numbers_float)
    
    return {
        'dien_tich': dien_tich,
        'chieu_dai': chieu_dai, 
        'chieu_rong': chieu_rong
    }



In [119]:
 
dien_tich_columns = [col for col in df.columns if 'dien_tich_dat' in col.lower() or 'area' in col.lower()]
if dien_tich_columns:
    dien_tich_col = dien_tich_columns[0]      
    tach_results = df[dien_tich_col].apply(tach_dien_tich_va_kich_thuoc)
    tach_df = pd.DataFrame(tach_results.tolist())
    
    df['dien_tich_dat_m2'] = tach_df['dien_tich']
    df['chieu_dai_m'] = tach_df['chieu_dai'] 
    df['chieu_rong_m'] = tach_df['chieu_rong']
    
    


In [91]:
df.head(2)

,dien_tich_su_dung_m2,dien_tich_dat,ma_bds,ngay_dang,nha_tam,phap_ly,phong_ngu,dia_chi,loai_hinh,mo_gioi_link,mo_gioi_phone,mo_gioi_ten,gia_ty,title,url,dien_tich_dat_m2,chieu_dai_m,chieu_rong_m
0,38 m2,"38 m2(3,5x12)",22658178,24/10/2025,3.0,Sổ đỏ,3.0,"Phố Vọng, Phường Đồng Tâm, Quận Hai Bà Trưng, ...",Mua bán nhà đất,https://mogi.vn/moi-gioi/0984857937-luong-tu-u...,984857937,Lương Tú,6 tỷ 500 triệu,"Bán Nhà hẻm, ngõ Phố Vọng",https://mogi.vn/quan-hai-ba-trung/mua-nha-hem-...,38.0,12.0,3.5
1,NaN,60 m2(5x14),22381248,24/10/2025,7.0,Sổ đỏ,6.0,"Hồ Tùng Mậu, Phường Mai Dịch, Quận Cầu Giấy, H...",Mua bán nhà đất,https://mogi.vn/moi-gioi/0919928661-phung-quan...,919928661,Phùng Quang Dậu,15 tỷ,"Bán Nhà mặt tiền, phố Hồ Tùng Mậu",https://mogi.vn/quan-cau-giay/mua-nha-mat-tien...,60.0,14.0,5.0


In [67]:
type(df['dien_tich_su_dung_m2'])

pandas.core.series.Series

In [120]:
df['dien_tich_su_dung_m2'] = df['dien_tich_su_dung_m2'].str.extract(r'(\d+(?:\.\d+)?)').astype(float)


In [121]:
def clean_room_data(value):
    if pd.isna(value):
        return None
    if isinstance(value, str):
        # Nếu là string như 'Nhà tắm', 'Phòng ngủ' thì chuyển thành NaN
        if value.lower() in ['nhà tắm', 'phòng ngủ', 'phong ngu', 'nha tam']:
            return None
        # Nếu là số dạng string, chuyển thành số
        try:
            return float(value)
        except:
            return None
    return value

# Làm sạch dữ liệu
df['nha_tam'] = df['nha_tam'].apply(clean_room_data)
df['phong_ngu'] = df['phong_ngu'].apply(clean_room_data)


# Phân tích mối quan hệ giữa diện tích và số phòng
# Tính trung bình số phòng theo diện tích
df_with_rooms = df[(df['nha_tam'].notna()) & (df['phong_ngu'].notna())].copy()
if len(df_with_rooms) > 0:
    # Tạo bins cho diện tích
    df_with_rooms['dien_tich_bin'] = pd.cut(df_with_rooms['dien_tich_su_dung_m2'], 
                                           bins=[0, 50, 80, 120, 200, float('inf')], 
                                           labels=['<50m²', '50-80m²', '80-120m²', '120-200m²', '>200m²'])
    
    # Tính trung bình số phòng theo từng bin diện tích
    avg_rooms_by_area = df_with_rooms.groupby('dien_tich_bin', observed=True).agg({
        'nha_tam': 'mean',
        'phong_ngu': 'mean'
    }).round(0)
    
    
    # fill dựa trên diện tích
    def fill_rooms_by_area(row):
        if pd.isna(row['nha_tam']) or pd.isna(row['phong_ngu']):
            area = row['dien_tich_su_dung_m2']
            if pd.isna(area):
                return row['nha_tam'], row['phong_ngu']
            
            # Xác định bin diện tích
            if area < 50:
                bin_name = '<50m²'
            elif area < 80:
                bin_name = '50-80m²'
            elif area < 120:
                bin_name = '80-120m²'
            elif area < 200:
                bin_name = '120-200m²'
            else:
                bin_name = '>200m²'
            
            # Fill từ trung bình của bin tương ứng
            if bin_name in avg_rooms_by_area.index:
                nha_tam_fill = avg_rooms_by_area.loc[bin_name, 'nha_tam'] if pd.isna(row['nha_tam']) else row['nha_tam']
                phong_ngu_fill = avg_rooms_by_area.loc[bin_name, 'phong_ngu'] if pd.isna(row['phong_ngu']) else row['phong_ngu']
                return nha_tam_fill, phong_ngu_fill
        
        return row['nha_tam'], row['phong_ngu']
    
    # Apply hàm fill
    df[['nha_tam', 'phong_ngu']] = df.apply(fill_rooms_by_area, axis=1, result_type='expand')
    
    # Fill các giá trị còn lại bằng trung bình chung
    if df['nha_tam'].isna().sum() > 0:
        df['nha_tam'] = df['nha_tam'].fillna(df['nha_tam'].mean())
    if df['phong_ngu'].isna().sum() > 0:
        df['phong_ngu'] = df['phong_ngu'].fillna(df['phong_ngu'].mean())
    
    # Convert về int - làm tròn trước khi convert
    df['nha_tam'] = df['nha_tam'].round().astype('Int64')
    df['phong_ngu'] = df['phong_ngu'].round().astype('Int64')

In [122]:
df['ma_bds'] = df['ma_bds'].astype(str)
df['nha_tam'] = pd.to_numeric(df['nha_tam'], errors='coerce').astype('Int64')
df['phong_ngu'] = pd.to_numeric(df['phong_ngu'], errors='coerce').astype('Int64')

In [139]:
dien_tich_su_dung_cols = [col for col in df.columns if 'dien_tich_su_dung' in col.lower()]
gia_cols = [col for col in df.columns if 'gia' in col.lower() and 'ty' in col.lower()]
quan_cols = [col for col in df.columns if 'quan' in col.lower() or 'huyen' in col.lower()]
loai_hinh_cols = [col for col in df.columns if 'loai_hinh' in col.lower() or 'loai' in col.lower()]


if dien_tich_su_dung_cols and gia_cols and quan_cols and loai_hinh_cols:
    dien_tich_col = dien_tich_su_dung_cols[0]
    gia_col = gia_cols[0]
    quan_col = quan_cols[0]
    loai_hinh_col = loai_hinh_cols[0]
    
 
    # Kiểm tra dữ liệu hiện tại
    nan_count = df[dien_tich_col].isna().sum()
    total_count = len(df)
   
    if nan_count > 0:
       
        gia_values = df[df[gia_col].notna()][gia_col]
      
        # Chuyển đổi cột giá sang numeric trước khi tạo bins
        df[gia_col] = pd.to_numeric(df[gia_col], errors='coerce')
        
        # Tạo 10 bins cho giá
        n_bins = 10
        df['gia_bin'] = pd.cut(df[gia_col], bins=n_bins, labels=False, include_lowest=True)
        
        # Hiển thị phân phối bins
        bin_counts = df['gia_bin'].value_counts().sort_index()
       
        
        # Tạo nhóm tương đồng: quận + loại hình + bin giá
        df['nhom_tuong_dong'] = df[quan_col].astype(str) + "_" + df[loai_hinh_col].astype(str) + "_" + df['gia_bin'].astype(str)
        
        # Xử lý NaN trong nhom_tuong_dong
        df['nhom_tuong_dong'] = df['nhom_tuong_dong'].fillna('unknown')
        
        # Tính trung bình diện tích theo nhóm tương đồng
        avg_by_group = df.groupby('nhom_tuong_dong')[dien_tich_col].mean()
        
        # Fill cho các dòng NaN có nhóm tương đồng
        fill_condition_1 = (
            df[dien_tich_col].isna() &
            df['nhom_tuong_dong'].isin(avg_by_group.index) &
            avg_by_group[df['nhom_tuong_dong']].notna()
        )
        
        fill_count_1 = fill_condition_1.sum()   
        if fill_count_1 > 0:
            # Fill diện tích - sửa lỗi mapping
            for idx in df[fill_condition_1].index:
                nhom = df.loc[idx, 'nhom_tuong_dong']
                if nhom in avg_by_group.index and pd.notna(avg_by_group[nhom]):
                    df.loc[idx, dien_tich_col] = avg_by_group[nhom]
            
           
            

ValueError: Bin edges must be unique: Index([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype='float64').
You can drop duplicate edges by setting the 'duplicates' kwarg

In [140]:
def chuan_hoa_gia(x):
    x = str(x).lower()
    ty = 0
    trieu = 0

    # Lấy phần 'tỷ'
    match_ty = re.search(r'(\d+(?:[\.,]\d+)?)\s*t[ỷi]', x)
    if match_ty:
        ty = float(match_ty.group(1).replace(',', '.'))

    # Lấy phần 'triệu'
    match_trieu = re.search(r'(\d+(?:[\.,]\d+)?)\s*triệu', x)
    if match_trieu:
        trieu = float(match_trieu.group(1).replace(',', '.'))

    # Đổi về tỷ đồng
    return round(ty + trieu/1000, 3)

def tach_dia_chi(dia_chi):
    dia_chi = str(dia_chi)
    
    # Tên đường (
    ten_duong = dia_chi.split(",")[0].strip()

    # Tên phường/xã
    phuong = re.search(r'Phường\s+([^,]+)', dia_chi)
    phuong = phuong.group(1).strip() if phuong else None

    # Tên quận/huyện
    quan = re.search(r'Quận\s+([^,]+)', dia_chi)
    quan = quan.group(1).strip() if quan else None

    # Thành phố / Tỉnh
    tp = re.search(r'(Thành phố|TP\.?|Tỉnh)\s+([^,]+)', dia_chi)
    tp = tp.group(2).strip() if tp else "Hà Nội"

    # Gộp tên đường + phường
    phuong_xa = f"{ten_duong} - Phường {phuong}" if phuong else ten_duong

    return pd.Series([phuong_xa, quan, tp])



def clean_dien_tich(value):
    if pd.isna(value):
        return None
    value = str(value).lower().strip()
    
    # Tìm số đầu tiên trong chuỗi (có thể có thập phân)
    match = re.search(r'(\d+(\.\d+)?)', value)
    if match:
        return float(match.group(1))
    else:
        return None

# lay ra loai hinhhinh
def lay_loai_hinh(title : str) -> str:
    t = (title or " ").lower()
    if re.search(r"\b(đất|đất thổ cư|lô đất|đất nền)\b", t):
        return "dat"
    elif re.search(r"\b(nhà |nhà riêng|nhà mặt|nhà tầng|nhà nghỉ|nhà đơn)\b", t):
        return "nha"
    elif re.search(r"\b(chung cư|chung cư nguyên căn|chung cư cao cấp|chung cư cao cấpchung cu cao cap)\b", t):
        return "chung_cu"
    else:
        return "mat_bang"

mask = df["dien_tich_su_dung_m2"].isna() & df["dien_tich_dat_m2"].notna()
df.loc[mask, "dien_tich_su_dung_m2"] = df.loc[mask, "dien_tich_dat_m2"]
df['loai_hinh'] = df['title'].apply(lay_loai_hinh)
df['dien_tich_dat_m2'] = df['dien_tich_dat_m2'].apply(clean_dien_tich)
df['gia(ty)'] = df['gia_ty'].apply(chuan_hoa_gia)
df[['phuong_xa', 'quan_huyen', 'thanh_pho']] = df['dia_chi'].apply(tach_dia_chi)

In [125]:
for c in ['phap_ly','phuong_xa','quan_huyen','thanh_pho']:
    df[c] = df[c].astype(str).str.strip()

In [126]:
df['ngay_dang'] = pd.to_datetime(df['ngay_dang'], format='%d/%m/%Y', errors='coerce')

In [144]:
df_tmp = df[['ma_bds','dien_tich_su_dung_m2','gia(ty)','title','phap_ly','ngay_dang','phuong_xa','quan_huyen','thanh_pho','nha_tam','phong_ngu','dien_tich_dat_m2','chieu_dai_m','chieu_rong_m','loai_hinh']]

In [128]:
df.head(1)

,dien_tich_su_dung_m2,dien_tich_dat,ma_bds,ngay_dang,nha_tam,phap_ly,phong_ngu,dia_chi,loai_hinh,mo_gioi_link,...,gia_ty,title,url,dien_tich_dat_m2,chieu_dai_m,chieu_rong_m,gia(ty),phuong_xa,quan_huyen,thanh_pho
0,38.0,"38 m2(3,5x12)",22658178,2025-10-24,3,Sổ đỏ,3,"Phố Vọng, Phường Đồng Tâm, Quận Hai Bà Trưng, ...",nha,https://mogi.vn/moi-gioi/0984857937-luong-tu-u...,...,6 tỷ 500 triệu,"Bán Nhà hẻm, ngõ Phố Vọng",https://mogi.vn/quan-hai-ba-trung/mua-nha-hem-...,38.0,12.0,3.5,6.5,Phố Vọng - Phường Đồng Tâm,Hai Bà Trưng,Hà Nội


In [129]:
df_tmp.head(1)

,ma_bds,dien_tich_su_dung_m2,gia(ty),title,phap_ly,ngay_dang,phuong_xa,quan_huyen,thanh_pho,nha_tam,phong_ngu,dien_tich_dat_m2,chieu_dai_m,chieu_rong_m,loai_hinh
0,22658178,38.0,6.5,"Bán Nhà hẻm, ngõ Phố Vọng",Sổ đỏ,2025-10-24,Phố Vọng - Phường Đồng Tâm,Hai Bà Trưng,Hà Nội,3,3,38.0,12.0,3.5,nha


In [145]:
df_tmp.isna().sum()

ma_bds                     0
dien_tich_su_dung_m2       1
gia(ty)                    0
title                      0
phap_ly                    0
ngay_dang                  0
phuong_xa                  0
quan_huyen               372
thanh_pho                  0
nha_tam                    0
phong_ngu                  0
dien_tich_dat_m2         636
chieu_dai_m             1095
chieu_rong_m            1095
loai_hinh                  0
dtype: int64

In [148]:
df_tmp[df_tmp['quan_huyen'].isna()].head(3)

,ma_bds,dien_tich_su_dung_m2,gia(ty),title,phap_ly,ngay_dang,phuong_xa,quan_huyen,thanh_pho,nha_tam,phong_ngu,dien_tich_dat_m2,chieu_dai_m,chieu_rong_m,loai_hinh
4,22698617,195.0,0.0,"Bán Nhà hẻm, ngõ Giáp Hải",Sổ đỏ,2025-10-24,Giáp Hải,None,Hà Nội,2,4,65.0,14.6,4.5,nha
9,22699693,355.0,0.0,"Bán Nhà mặt tiền, phố Bát Khối",Sổ đỏ,2025-10-24,Bát Khối,None,Hà Nội,3,4,117.0,19.0,6.2,nha
13,22700445,400.0,0.0,"Bán Nhà hẻm, ngõ Ngô Xuân Quảng",Sổ đỏ,2025-10-24,Ngô Xuân Quảng,None,Hà Nội,6,15,100.0,16.5,6.0,nha


4
